<a href="https://colab.research.google.com/github/GenAIHub/genai-workshop/blob/main/03_Agents/04_react_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct Agent with Query Engine (RAG) Tools

In this section, we show how to setup an agent powered by the ReAct loop for financial analysis.

The agent has access to two "tools": one to query the 2021 Lyft 10-K and the other to query the 2021 Uber 10-K.

## Build Query Engine Tools

In [ ]:
!pip install boto3
!pip install llama-index
!pip install llama-index-llms-bedrock
!pip install llama-index-embeddings-bedrock

In [9]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [10]:
import os
import boto3

# Set AWS env config
region_name = os.getenv("AWS_REGION", "us-east-1")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID", "")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY", "")

In [11]:
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import Settings

llm_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
embed_model_id = "amazon.titan-embed-text-v1"

llm = Bedrock(
    model=llm_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    temperature=0.1,
    max_tokens=512
    )

embed_model = BedrockEmbedding(
    model=embed_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    )

Settings.llm = llm
Settings.embed_model = embed_model

Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [13]:
# load data
lyft_docs = SimpleDirectoryReader(
    input_files=["./data/10k/lyft_2021.pdf"]
).load_data()
uber_docs = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()

# build index
lyft_index = VectorStoreIndex.from_documents(lyft_docs)
uber_index = VectorStoreIndex.from_documents(uber_docs)


In [14]:
# build query engines
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup ReAct Agent

You can **optionally** specify context which will be added to the core ReAct system prompt.

In [15]:
from llama_index.core.agent import ReActAgent

In [16]:
# [Optional] Add Context
# context = """\
# You are a stock market sorcerer who is an expert on the companies Lyft and Uber.\
#     You will answer questions about Uber and Lyft as in the persona of a sorcerer \
#     and veteran stock market investor.
# """

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [17]:
response = agent.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question about Lyft's revenue growth in 2021.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Based on the information provided, Lyft's revenue increased in 2021 compared to 2020, reflecting the recovery in demand for its platform as more communities reopened and travel restrictions eased during the COVID-19 pandemic recovery. However, the exact revenue growth percentage for 2021 is not explicitly stated.

The context mentions that revenue per Active Rider (a key metric for Lyft) increased in each quarter of 2021 compared to the same periods in 2020, driven by factors like improvement in demand, increase in ride frequency, and shift towards higher revenue rides like airport trips. It also notes that revenues benefited from licensing and data access agreements starting in Q2 2021.

While the specific revenue growth figure is not provided, the

## Run Some Example Queries

We run some example queries using the agent, showcasing some of the agent's abilities to do chain-of-thought-reasoning and tool use to synthesize the right answer.

We also show queries.

In [18]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

Thought: The current language of the user is: English. I need to use the tools to get information on Uber and Lyft's revenue growth in 2021 to compare and analyze them.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth percentage in 2021 compared to 2020?"}
Observation: Based on the information provided, Lyft's revenue increased by 36.0% in 2021 compared to 2020. The context states that revenue in 2021 was $3.2 billion, while revenue in 2020 was $2.4 billion, representing a 36.0% year-over-year increase.
Thought: I now have information on Lyft's revenue growth in 2021. To compare and contrast with Uber, I need to get Uber's revenue growth information as well.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth percentage in 2021 compared to 2020?"}
Observation: According to the information provided, Uber's revenue increased by 57% in 2021 compared to 2020.
Thought: I now have the revenue growth percentages for both Lyft and Uber in 2021. I ca

In [19]:
response = agent.chat(
    "Can you tell me about the risk factors of the company with the higher"
    " revenue?"
)
print(str(response))

Thought: To analyze the risk factors of the company with higher revenue in 2021 (which was Uber based on the previous analysis), I will need to use the uber_10k tool.
Action: uber_10k
Action Input: {'input': 'What were the major risk factors facing Uber in 2021?'}
Observation: Based on the context information provided, some of the major risk factors facing Uber in 2021 included:

1. Potential impact of the COVID-19 pandemic on demand for Uber's mobility services and overall business operations. The pandemic led to a significant drop in demand for ride-hailing services due to lockdowns and travel restrictions.

2. Legal challenges and lawsuits related to the classification of drivers as independent contractors rather than employees in various jurisdictions. Uber was involved in numerous legal proceedings globally regarding this issue.

3. Safety incidents and criminal activities involving drivers, consumers, or third parties on Uber's platform, which could harm the company's reputation 